In [ ]:
import numpy as np

x = np.load('features.npy')
lis = []
for i,j in enumerate(x):
    lis.append((i,j))

In [ ]:
print(lis[0])
new_lis = []

for x in lis:
    new_lis.append(np.mean(x[1],axis=0))
print(new_lis[0])

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
import matplotlib
import json

linked = linkage(np.asarray(new_lis), 'single')
with open('../dataset/something-something-v2-labels.json') as f:
    reads = f.read()
    x = json.loads(reads)
rev_id = {}
for a,b in x.items():
    rev_id[b] = a
labelList = [a for a,b in x.items()]

matplotlib.rc('xtick', labelsize=200) 
matplotlib.rc('ytick', labelsize=20)
ax = plt.figure(figsize=(60, 40))
#ax = plt.subplots()
#ax.tick_params(axis='both', which='major', labelsize=10) 
#ax.tick_params(axis='both', which='minor', labelsize=8)
dendrogram(linked,
            orientation='top',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)
fig = plt.gcf()
plt.savefig('foo.png', dpi=600)
#plt.show()
#plt.savefig('clusters.png')
#plt.savefig('foo.png', bbox_inches='tight')


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from collections import defaultdict

cluster = AgglomerativeClustering(n_clusters=48, affinity='euclidean', linkage='ward')
cluster.fit_predict(np.asarray(new_lis))
print(cluster.labels_)

dic = defaultdict(list)
print(dic)
#print(rev_id)
for i,k in enumerate(cluster.labels_):
    dic[int(k)].append(rev_id[str(i)])
for x,y in dic.items():
    print(y)

In [ ]:
import scipy
def cos_cdist(vector):
    # getting cosine distance between search image and images database
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(new_lis, v, 'cosine').reshape(-1)

for i,f in enumerate(new_lis):
    img_distances = cos_cdist(f)
    nearest_ids = np.argsort(img_distances)[:5].tolist()
    #print(rev_id[str(i)])
    lis = [rev_id[str(x)] for x in nearest_ids]
    print(lis)
    

In [ ]:
import h5py
import numpy as np
import deepdish as dd
abc = np.asarray([1,2,3],dtype=np.float32)
dic = {'14324':[1,[[1],2,3,4]],'1212':[2,[1,2]],'11':abc}
s = str(dic)
print(s)
dd.io.save('test.h5', {'data': dic}, compression=None)

#with h5py.File('feat.hdf5','w') as f:
#    f.create_dataset(s,dtype="f")

In [ ]:
'''import ast 
h = h5py.File('feat.hdf5','r')
for x in h:
    print(x[:100])
    res = ast.literal_eval(x)
    print(x)
print(type(res))
for k,v in res.items():
    print(v)
h.close()'''
x = dd.io.load('feats.h5')
print(type(x['data']))
print(x['data'][1][1][0].shape)

In [ ]:
import numpy as np
x = np.load('features.npy',allow_pickle=True)
print(x.shape)
for y in x:
    print(y[1].shape)

In [ ]:
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.optimizers import SGD
import numpy as np
x = np.load('features.npy',allow_pickle=True)

'''def define_model(in_shape=(8,2048), out_shape=174):
    model = Sequential()
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'),input_shape=in_shape)
    model.add(Dense(out_shape, activation='softmax'))
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['categorical_accuracy'])
    return model

model = define_model()'''
x = x[:100,]
y = [a[0] for a in x]
x = [a[1] for a in x]
#print(y,x)
#history = model.fit_generator(train_it, steps_per_epoch=len(x), epochs=200, verbose=0)
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
import torch

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(16384,2048)
        self.output = nn.Linear(2048, 174)
        
        # Define sigmoid activation and softmax output 
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        #print(x.size())
        
        return x
net = Network()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for data in zip(x,y):
        inputs, labels = data
        labels = np.asarray([0 if x!= labels else 1 for x in range(174)])
        #print(labels)
        inputs = inputs.reshape(-1)
        inputs = Variable(torch.from_numpy(inputs))
        labels = Variable(torch.from_numpy(labels))
        optimizer.zero_grad()
        outputs = net(inputs)
        print(outputs.size(),labels.size())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

print('Finished Training')